In [2]:
import numpy as np
import pandas as pd

import scipy.stats as sts

In [13]:
ll = pd.read_csv('processed_data/df_ll.csv')

KeyboardInterrupt: 

In [6]:
ll = ll.drop('LENDERS_lst',axis=1)

In [14]:
ll.head()

,LOAN_ID,lender_v,value
0,483693,970524,1
1,483693,1153379,1
2,483693,187221,1
3,483693,758772,1
4,483693,1090160,1


In [10]:
ll.to_csv('ll_ones.csv',index=False)

In [25]:
len(np.unique(ll['LOAN_ID']))

1387432

In [26]:
len(np.unique(ll['lender_v']))

1383799

In [24]:
df_array = ll.values

In [16]:
df_array[0]

array([483693, 970524,      1])

In [18]:
from scipy.sparse import csr_matrix

In [38]:
df_sparse = csr_matrix( (df_array[:,2], (df_array[:,0], df_array[:,1])), shape=(1450000,1387432))

In [39]:
df_sparse

<1450000x1387432 sparse matrix of type '<class 'numpy.int64'>'
	with 27459086 stored elements in Compressed Sparse Row format>

In [41]:
matrix_size = df_sparse.shape[0]*df_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(df_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

99.9986350826066

In [42]:
import numpy as np

from lightfm.datasets import fetch_movielens

data = fetch_movielens(min_rating=5.0)

/anaconda3/lib/python3.6/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [46]:
from lightfm import LightFM

In [47]:
data['train']

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 19048 stored elements in COOrdinate format>

In [48]:
model = LightFM(loss='warp')
%time model.fit(df_sparse, epochs=5, num_threads=2)

CPU times: user 11min 39s, sys: 14.9 s, total: 11min 54s
Wall time: 20min 37s


In [12]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  